In [229]:
#Code which generates the relation graph with both broken ways, and original ways
#Keep in mind, we are going to ignore the original ways which were broken down. We need to filter them.

import json
import networkx as nx
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import folium
import polyline

In [151]:

# Load data
with open('way_objects.json') as fp:
    way_objects = json.load(fp)

with open('broken_way_objects.json') as fp:
    broken_way_objects = json.load(fp)

keyset = list(way_objects.keys())
print("Original keys: ",len(keyset) )
print("Broken keys: ",len(broken_way_objects.keys()) )

for newkey in broken_way_objects.keys():
    original_key = newkey.split("_")[0]
    if(original_key in keyset):
        keyset.remove(original_key)
    keyset.append(newkey)

#lets combine two objects now
broken_way_objects.update(way_objects)
#lets remove keys which we do not need now
print("All keys after merging: ",len(broken_way_objects.keys()) )

with open('way_metrics.json') as fp:
    way_metrics = json.load(fp)

# ^ this will determine the final keyset
new_ways = {}
for key in list(way_metrics.keys()):
    new_ways[key] = broken_way_objects[key]

print("Num of keys in final object: ",len(new_ways.keys()) )




Original keys:  219
Broken keys:  789
All keys after merging:  1008
Num of keys in final object:  336


In [152]:
print(len(set(new_ways.keys())))

336


In [153]:

G = nx.DiGraph()

# Define a function to check if two points are within 10 meters
def is_within_distance(point1, point2, distance_m=10):
    return geodesic(point1, point2).meters <= distance_m

way_relations = {}
not_first_ways = [] #stores all the ways which are never first i.e children

# Create nodes for each way ID
for wayid in new_ways:
    way_relations[wayid] = []
    G.add_node(wayid)

# Create edges based on proximity
for wayid, coordinates in new_ways.items():
    end_point = tuple(coordinates[-1])
    for other_wayid, other_coordinates in new_ways.items():
        if wayid != other_wayid:
            start_point_other = tuple(other_coordinates[0])
            if is_within_distance(end_point, start_point_other):
                way_relations[wayid].append(other_wayid)
                not_first_ways.append(other_wayid)
                G.add_edge(wayid, other_wayid)



In [154]:
def find_root_nodes(G):
    """Find all nodes in the graph G that have no parents."""
    return [node for node in G if G.in_degree(node) == 0]

def find_leaf_nodes(G):
    """Find all nodes in the graph G that have no children."""
    return [node for node in G if G.out_degree(node) == 0]

def find_paths(G, start_node, end_nodes, path=[]):
    """Find all paths from start_node to any of the end_nodes, without revisiting nodes."""
    path = path + [start_node]
    if start_node in end_nodes:
        return [path]
    paths = []
    for node in G.successors(start_node):
        if node not in path:
            newpaths = find_paths(G, node, end_nodes, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths
    


In [200]:
#lets get the list of ways which mark the beginning of the tree
all_ways = set(new_ways.keys())
not_first_ways = set(not_first_ways)
first_ways = all_ways - not_first_ways
print("All the different ways in the map: ", len(all_ways))
print("All the ways which have a parent: ", len(not_first_ways))
print("All the ways which do not have a parent: " , len(first_ways)) 

first_way_objects = {x: new_ways[x] for x in new_ways.keys() if x  in first_ways}

with open('first_way_objects_after_break.json', 'w+') as fp:
    json.dump(first_way_objects, fp)

avg_lat = sum(coord[0] for  wayid, coordinates in first_way_objects.items() for coord in coordinates) / sum(len(coordinates) for wayid, coordinates in first_way_objects.items())
avg_lon = sum(coord[1] for  wayid, coordinates in first_way_objects.items() for coord in coordinates) / sum(len(coordinates) for wayid, coordinates in first_way_objects.items())
map_osm = folium.Map(location=[avg_lat, avg_lon], zoom_start=14)


way_objects = {}
for wayid, coordinates in new_ways.items():
    # Extract the coordinates for the way
    # Create a polyline with the coordinates and add to the map
    
    #storing it in the temporary database
    # way_objects[wayid] = coordinates

    folium.PolyLine(coordinates, color="orange", weight=2.5, opacity=1).add_to(map_osm)
    # Optionally, add a marker for the first node of the way with a popup for the way ID
    folium.Marker(
        [coordinates[0][0], coordinates[0][1]],
        popup=f"Start Way ID: {wayid}"
    ).add_to(map_osm)


# Save to an HTML file
output_file = './osm_new_way_after_break_visualization.html'
map_osm.save(output_file)


All the different ways in the map:  336
All the ways which have a parent:  266
All the ways which do not have a parent:  70


In [156]:
# Assuming G is your DiGraph
root_nodes = find_root_nodes(G)
leaf_nodes = find_leaf_nodes(G)

print(len(root_nodes))

70


In [168]:
import networkx as nx
from geopy.distance import geodesic

def calculate_distance_elevation(wayid):
    """
    Calculate the total distance and elevation gain of a way.
    way_coordinates: List of (latitude, longitude) tuples.
    elevations: List of elevations corresponding to each coordinate.
    """
    total_distance = way_metrics[wayid]['distance']  # Total distance in meters
    total_elevation_gain = way_metrics[wayid]['gain']  # Total elevation gain in meters

    return [total_distance, total_elevation_gain]

def dfs_find_trails(G, start_node, target_distance, target_elevation_gain, current_path=[], current_distance=0, current_elevation=0, distance_tolerance=50):
    # Add the start_node to the current path
    current_path = current_path + [start_node]

    # Check if the current path meets the distance and elevation criteria
    if current_distance >= target_distance and current_distance <= target_distance + distance_tolerance and current_elevation >= target_elevation_gain:
        return [current_path]

    paths = []
    for node in G.successors(start_node):
        # Check if the node has already been visited in the current path
        if node not in current_path: 
            # Calculate the distance and elevation for the new way
            way_distance, way_elevation = calculate_distance_elevation(node)
            new_distance = current_distance + way_distance
            new_elevation = current_elevation + way_elevation

            # Continue the search if the new distance and elevation are within tolerance
            if new_distance <= target_distance + distance_tolerance: 
                newpaths = dfs_find_trails(G, node, target_distance, target_elevation_gain, current_path, new_distance, new_elevation, distance_tolerance)
                paths.extend(newpaths)

    return paths



In [165]:
# Assuming G is your DiGraph and new_ways contains way data
root_nodes = [node for node in G if G.in_degree(node) == 0]
print(root_nodes)
print(len(root_nodes))


# Now all_trails contains paths that meet the criteria

['36006153_0', '59878931_1', '73579855_0', '73579864_0', '73579928_0', '73622155_0', '73680752_2', '73680766_0', '73680792_0', '73680802_2', '73680807_0', '73680820_0', '73680845_2', '73680850_0', '73680855_0', '73680863_1', '73680875_0', '73680877_1', '73680902_0', '73680929_1', '73680951_1', '73680955_0', '73681015_0', '73681079_0', '73681083_0', '97309332_0', '97918391_0', '105107028_1', '113660806_0', '116417122_0', '129859895_1', '198354140_0', '264409328_0', '311287374_1', '342856328_0', '342856334_1', '342860382_1', '342860611_0', '343270417_1', '361823723_0', '361823725_1', '364951580_0', '400679135_0', '437190620_0', '442967665_0', '442967666_0', '471348777_0', '517774838_0', '521978877_0', '522322511_0', '544660536_0', '837711407_0', '937982930_0', '963610548_1', '963610550_1', '963610552_0', '963610553_0', '963610554_0', '963610557_0', '963610558_0', '993367635_0', '996186621_0', '996186629_0', '997648059_0', '1008955114_0', '1149770527_0', '1149770528_0', '1149772390_0', '1

In [213]:
all_trails = []
distance_in_meters = 7000
vert_in_meters = 300
for root in range(len(root_nodes)):
    all_trails.extend(dfs_find_trails(G, root_nodes[root], distance_in_meters, vert_in_meters, distance_tolerance=100))

print(len(all_trails))

8


In [143]:
# all_trails = [all_trails[0]]
# all_trails

[['36006153_0',
  '36006153_1',
  '36006153_2',
  '442967671_0',
  '442967671_1',
  '73681061_2',
  '36006153_4',
  '36006153_5',
  '377893257_0',
  '377893257_1']]

In [230]:

print(len(all_trails))
# Create a Folium map
# Initialize the map with a central location
# map_center = [first_line.coords[0][0], first_line.coords[0][1]]

map_center = [40.014984,-105.270546 ]
m = folium.Map(location=map_center, zoom_start=13)

# Function to add a trail to the map
def add_trail_to_map(trail_coordinates, map_object):
    # trail is a list of coordinates [(lat, lon), (lat, lon), ...]
        # Extract start and end coordinates of the trail
    start_coord = trail_coordinates[0]  # First coordinate of the first way
    end_coord = trail_coordinates[-1]  # Last coordinate of the last way

    # Add markers for start and end points
    folium.Marker(
        start_coord,
        popup="Start",
        icon=folium.Icon(color='green', icon='play')
    ).add_to(map_object)

    folium.Marker(
        end_coord,
        popup="End",
        icon=folium.Icon(color='red', icon='stop')
    ).add_to(map_object)


    folium.PolyLine(trail_coordinates, color='blue', weight=2.5, opacity=1).add_to(map_object)


polylines = []
coordinates = []
# Add each trail to the map
for trail_set in all_trails:  # Assuming all_trails is a list of trails
    trail_coordinates = []
    for each_trail in trail_set:
        trail_coordinates.extend(new_ways[each_trail])  # Convert way IDs to coordinates


    #printout info about that trail
    dist,ele = get_distance_elevation(trail_coordinates)
    # geojson_polyline = coordinates_to_geojson(coordinates)

    coordinates.append(trail_coordinates)
    polylines.append(geojson_polyline)
    print("Trail Distance: {} Trail Elevation: {}".format(dist, ele))

    
    add_trail_to_map(trail_coordinates, m)

# Save or display the map
m.save('trails_map.html')


8
Trail Distance: 9279.401337399913 Trail Elevation: 749.0
Trail Distance: 9279.401337399913 Trail Elevation: 749.0
Trail Distance: 9348.65896829038 Trail Elevation: 767.0
Trail Distance: 9348.65896829038 Trail Elevation: 767.0
Trail Distance: 9480.83444280033 Trail Elevation: 750.0
Trail Distance: 9480.83444280033 Trail Elevation: 750.0
Trail Distance: 9550.09207369079 Trail Elevation: 768.0
Trail Distance: 9550.09207369079 Trail Elevation: 768.0


'{"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "LineString", "coordinates": [[39.9929181, -105.2816038], [39.9932671, -105.2815745], [39.9933664, -105.2814224], [39.9935854, -105.2812437], [39.9937912, -105.2811471], [39.9941659, -105.2808223], [39.9943909, -105.2807667], [39.9945379, -105.2805338], [39.9947305, -105.2804174], [39.9948349, -105.2804597], [39.9949262, -105.2803631], [39.9950255, -105.2803869], [39.9951573, -105.28024], [39.9953175, -105.2802586], [39.9955391, -105.2800253]]}, "properties": {}}]}'

In [196]:
def get_distance_elevation(way_coordinates):
    total_distance = 0  # Total distance in meters
    total_elevation_gain = 0

    for i in range(len(way_coordinates) - 1):
        # Calculate distance between consecutive points
        point1 = way_coordinates[i]
        point2 = way_coordinates[i+1]

        distance = geodesic(point1, point2).meters
        total_distance += distance

    
    point1 = way_coordinates[0]
    point2 = way_coordinates[-1]


    point1_elevation = 0
    point2_elevation = 0

    api_url = f"https://api.open-elevation.com/api/v1/lookup?locations={point1[0]},{point1[1]}"
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        # Extract elevation from the response
        point1_elevation = data['results'][0]['elevation']
    else:
        point1_elevation = 0

    api_url = f"https://api.open-elevation.com/api/v1/lookup?locations={point2[0]},{point2[1]}"
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        # Extract elevation from the response
        point2_elevation = data['results'][0]['elevation']
    else:
        point2_elevation = 0

    total_elevation_gain += point2_elevation - point1_elevation
    return [total_distance,total_elevation_gain]

In [235]:
encoded_polyline = polyline.encode(coordinates[0])
js_coordinates = '[' + ', '.join(['[' + str(lon) + ', ' + str(lat) + ']' for lat, lon in coordinates[0]]) + ']'
print(js_coordinates)

[[-105.2801988, 39.9972788], [-105.2801228, 39.9971888], [-105.28014, 39.9971319], [-105.2802749, 39.9969655], [-105.2803532, 39.9967215], [-105.2803374, 39.9966522], [-105.2801949, 39.9965507], [-105.2801949, 39.9965507], [-105.2802179, 39.9964875], [-105.2802408, 39.9964173], [-105.280293, 39.996355], [-105.2803594, 39.996283], [-105.2804159, 39.9961851], [-105.280508, 39.9960399], [-105.2806459, 39.9957939], [-105.2808521, 39.995585], [-105.2811248, 39.9954084], [-105.2812848, 39.9953085], [-105.2815101, 39.9950104], [-105.2815462, 39.9949574], [-105.2815462, 39.9949574], [-105.2815742, 39.9949123], [-105.2816659, 39.9947327], [-105.2817974, 39.9941009], [-105.2821653, 39.9934318], [-105.282171, 39.9930506], [-105.2822742, 39.9928774], [-105.2822742, 39.9928774], [-105.2822477, 39.9928888], [-105.2819825, 39.9929804], [-105.2818777, 39.9928337], [-105.2816038, 39.9929181], [-105.2816038, 39.9929181], [-105.2814228, 39.9929738], [-105.2811366, 39.992986], [-105.2811366, 39.992986], [

In [ ]:
####EXPERIMENTATION AREA AFTER THIS

In [210]:
## experiment code to get all the trails possible

def dfs_find_all_trails(G, start_node, current_path=[]):
    # Add the start_node to the current path
    current_path = current_path + [start_node]

    # If no more successors, return the current path
    if len(list(G.successors(start_node))) == 0:
        return [current_path]

    paths = []
    for node in G.successors(start_node):
        # Check if the node has already been visited in the current path to avoid cycles
        if node not in current_path: 
            newpaths = dfs_find_all_trails(G, node, current_path)
            paths.extend(newpaths)

    return paths

# Assuming G is your graph
all_trails = []
for start_node in G.nodes():
    all_trails.extend(dfs_find_all_trails(G, start_node))

# all_trails now contains all possible trails in the graph
print(len(all_trails))


6634
